In [1]:
import sciris as sc
import pandas as pd
import covasim as cv
import covasim.parameters as cvp
import pylab as pl
import numpy as np
import os
import math
import random
import dask
from dask.distributed import Client

Covasim 3.1.4 (2022-10-22) — © 2020-2022 by IDM


In [2]:
# set working directory
new_directory = 'C:/Users/irisg/Documents/PhD/COVID_France/SEIR_vs_Rt_sims/ABM_2params_all_at_once10'
os.chdir(new_directory)

In [3]:
def simulate_rm(params):
    beta, pop, initE, index = params
    pars = dict(
        pop_size=pop, pop_infected=initE, beta=beta,
        pop_type='random', location='France',
        start_day='2020-02-01', end_day='2020-06-30',
        use_waning=False, prog_by_age = False,
        pop_scale = 10, rescale = True, 
        asymp_factor = 1,
        prognoses = dict(age_cutoffs = np.array([0, 50]),
                         sus_ORs = np.array([1, 1]),
                         trans_ORs = np.array([1, 1]),
                         comorbidities = np.array([1, 1]),
                         symp_probs = np.array([1, 1]),
                         severe_probs = np.array([0.1, 0.1]),
                         crit_probs = np.array([1, 1]),
                         death_probs = np.array([0.1, 0.1]))
    )
    
    # transmission reduction of NPI 1: exp(-1.45) = 0.2345703
    NPI1 = cv.change_beta(days='2020-03-17', changes=0.2345703)
    # transmission reduction of NPI 2: exp(-0.8) = 0.449329
    NPI2 = cv.change_beta(days='2020-05-11', changes=0.449329)
    
    # no testing structure
    tests = cv.test_prob(symp_prob = 0, asymp_prob = 0, start_day = '2020-02-01')

    # run simulation
    sim = cv.Sim(pars, interventions=[NPI1, NPI2, tests], label= f'Sim {index}')
    sim.run()

    # return  observations and Rt
    return sim.results['r_eff'].values[1:151], sim.results['new_infectious'].values[1:151], sim.results['new_severe'].values[1:151], sim.results['new_deaths'].values[1:151], sim.results['n_severe'].values[1:151]

In [14]:
# simulation settings: population size, number of regions and time frame, number of repetitions
popsize_df = pd.read_csv('C:/Users/irisg/Documents/PhD/COVID_France/Dropbox_iris_covid/departement/simulations/Rt_trajectories/popsize_df.csv')
popsizes = popsize_df['popsize']
ps = popsizes.values

depts = np.arange(1, 95)
days = np.arange(1, 151)
random_seed = np.arange(100, 201)
reps = np.arange(1, 101)

In [15]:
if __name__ == '__main__':

    # Run settings
    n_workers = 12
    
    # Create and queue the Dask jobs
    client = Client(n_workers=n_workers)
    
    for rep in reps:
        
        rm_sims = []
       
        # randomly sample starting parameters
        np.random.seed(random_seed[rep])
        betas = np.random.lognormal(math.log(0.014), 0.04, size=94)
        initEs = np.random.normal(50, 10, size=94)
        
        index = np.arange(1, 95)
   
        # set simulation for all departments
        for beta, pop, initE, i in zip(betas, popsizes, initEs, index):
            params_list = [beta, pop, initE, i]
            sim = dask.delayed(simulate_rm)(params_list)
            rm_sims.append(sim)

        # Run and process the simulations
        res = list(dask.compute(*rm_sims))
        
        # Gather the simulated time series into a dataframe 
        Rt_list = []
        IncI_list = []
        IncH_list = []
        IncD_list = []
        PrevH_list = []
        for item in res:
            Rt_list.extend(item[0])
            IncI_list.extend(item[1])
            IncH_list.extend(item[2])
            IncD_list.extend(item[3])
            PrevH_list.extend(item[4])

        # Create a DataFrame
        rm_Inc_df = pd.DataFrame({'Rt': Rt_list, 'IncI': IncI_list, 'IncH': IncH_list, 'IncD': IncD_list, 'PrevH': PrevH_list})

        
        # create other rows necessary for Monolix estimation and scale observations to 10,000 population
        rm_Inc_df['dept_id'] = np.repeat(depts, 150)
        rm_Inc_df['day'] = np.tile(days, 94)
        rm_Inc_df['lockdown1'] = np.where((rm_Inc_df['day'].between(45, 99)), 1, 0)
        rm_Inc_df['BG1'] = np.where(rm_Inc_df['day'] >= 100, 1, 0)
        rm_Inc_df['popsize'] = np.repeat(ps, 150)
        
        rm_Inc_df = rm_Inc_df[['dept_id', 'day', 'IncI', 'IncH', 'IncD', 'PrevH', 'Rt', 'lockdown1', 'BG1', 'popsize']]

        # save data set
        rm_Inc_df.to_csv(f'data_covasim_rm10_Rt_{rep}.csv', index=False, sep=',')

C:\Users\irisg\anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52591 instead
  warnings.warn(
